In [1]:
import sys
import time
import requests
from datetime import datetime, timedelta
import json
import fasttext
import re
from bs4 import BeautifulSoup
import lxml
import numpy as np
import os

MAX_RETRIEVED_ELEMENTS = 1000
PUSHSHIFT_ENDPOINT = 'https://api.pushshift.io/reddit/search/'
THRESHOLD = 0.8
PATTERN = re.compile("&gt;.*?(\n|\s\s)")
HTTP_PATTERN = re.compile(r'[\(\[]?https?:\/\/.*?(\s|\Z)[\r\n]*')
EARLIEST_DATE = datetime(2019, 12, 19)
DIRNAME = 'state_analysis/'
PRETRAINED_MODEL_PATH = 'lid.176.bin'
model = fasttext.load_model(PRETRAINED_MODEL_PATH)

In [77]:
# HTTP_PATTERN = re.compile(r'[\(\[]?https?:\/\/.*?(\s|\Z)[\r\n]*')

# reddit_url = 'this is a [https://www.reddit.com/r/LocationReddits/wiki/faq/northamerica] and i think https://www.reddit.com/r/LocationReddits/wiki/faq/northameric'

# hp = 'ra bababa bab'

# re.sub(HTTP_PATTERN, "LINK ", reddit_url)
# # re.sub(r"[ab].*?\s", "LINK", hp)

'this is a LINK and i think LINK '

In [2]:
reddit_url = 'https://www.reddit.com/r/LocationReddits/wiki/faq/northamerica'
page = requests.get(reddit_url, headers = {'User-Agent': 'Mozilla/5.0'})
html = BeautifulSoup(page.text, "lxml")

In [42]:
# p = ['Alabama', 'New Jersey', "Maine", "Ba"]
# for s in p:
#     if len(re.findall(r"[M-Z]", s)) > 0:
#         print(s)

New Jersey
Maine


In [3]:
states = html.find_all("h5")[13:]
state_sub_map = {}
for state in states:
    if state.text == 'California':
        state_sub_map[state.text] = ['california']
        continue
#     if len(re.findall(r"[N-Z]", state.text)) <= 0:
#         continue
    links = state.findNext().find_all('a')
    links = [link['href'][3:] for link in links]
    state_sub_map[state.text] = links


In [4]:
cali = html.find_all("h6")
for part in cali:
    links = part.findNext().find_all('a')
    links = [link['href'][3:] for link in links]
    state_sub_map['California'].extend(links)
    


In [29]:
def none_or_empty(text):
    return text is None or len(text) == 0 or text == "[removed]" or text == '[deleted]'

def preprocess(text):
    # Need to get rid of &gt for quotes
    preproc = PATTERN.sub("", text)
    preproc = HTTP_PATTERN.sub("LINK ", preproc)
    # Need to get rid of newlines for fasttext to work
    preproc = re.sub("\n", " ", preproc)
    return preproc

In [3]:
query = PUSHSHIFT_ENDPOINT + 'comment/?q=coronacoaster+continues'
r = requests.get(query)
x = r.json()['data']
# ele  = [ele for ele in x if ele['author'] == 'Duese'][0]['body']
# ele = ele + "[https://www.reddit.com/r/LocationReddits/wiki/faq/northamerica]"
x

[{'all_awardings': [],
  'associated_award': None,
  'author': 'LilPeanutt',
  'author_flair_background_color': None,
  'author_flair_css_class': None,
  'author_flair_richtext': [],
  'author_flair_template_id': None,
  'author_flair_text': None,
  'author_flair_text_color': None,
  'author_flair_type': 'text',
  'author_fullname': 't2_2nrkew3e',
  'author_patreon_flair': False,
  'author_premium': False,
  'awarders': [],
  'body': "Editorial cartoon by Bill Bramhall, May 11, 2020.\n\n[Link](https://www.nydailynews.com/opinion/ny-bramhall-editorial-cartoons-2020-20200107-7ctvclbkyng5nl3ocyeet2cax4-photogallery.html)\n\nThe coronacoaster continues  🎢 \n\n[GoLocalProv](https://www.golocalprov.com/news/19-new-ri-coronavirus-deaths-highest-in-5-days-watch-saturday-1-pm-briefing)link from 5/9/2020\n\n&gt;On Saturday, Rhode Island Governor lifted the stay at home order in effect in Rhode Island. \n\n**EDITOR'S NOTE: Raimondo does not allow Rhode Islanders to hear reporters' questions. Near

In [27]:
# preprocess(ele)

"WOOTheir population is healthier than ours, albeit older. However, given the rate of obesity in america and obesity being a major risk factor for covid 19, it doesnt take an epidemiologist to realize we're in trouble.  Everything you just said was completely wrong and misleading.  I'm not trying to be a dick here, but you are completely misinformed across the board here.    First and foremost, in order to get to the conclusions you are drawing you have to blatantly ignore everything we know about mortality of the virus, the details of the deaths in Italy and the impact of secondary factors in those deaths.    Let's just point out the obvious here but you just tried to equate two countries that have a SEVEN YEAR difference in average age.  When one of the most direct effects on the mortality rate of the virus is based on age, I don't know how you can ignore this or presume that they are the same.  But you ignored that or tried to marginalize it which you can't do if you want to present

In [30]:
def valid_text(text):
    if none_or_empty(text):
        return False, ""
    preproc_text = preprocess(text)
    labels, prob = model.predict(preproc_text)
    if labels[0].endswith('en') and prob[0] >= THRESHOLD:
        return True, preproc_text
    return False, ""
    

In [31]:
def get_dates(latest_date):
    # Get all week increments from earliest date until present
    dates = []
    epochs = []
    curr = EARLIEST_DATE
    while curr < latest_date:
        dates.append(curr.strftime("%m/%d/%Y"))
        epochs.append(int(curr.timestamp()))
        curr += timedelta(weeks=1)
    dates.append(latest_date.strftime("%m/%d/%Y"))
    epochs.append(int(latest_date.timestamp()))
    return dates, epochs

In [36]:
def retrieve_subreddit_data(subreddit, retrieval_type, dirname, fields, keywords, dates):
#     print(len(dates))
    today = datetime.utcnow()
    today_timestamp = int((today - datetime(1970, 1, 1)).total_seconds())
    before_date = today_timestamp
    
    fields = ",".join(fields)
    query_words = "|".join(keywords)
    text_tag = 'body' if retrieval_type == 'comment' else 'selftext'
   
    count = 0
    total_valid = 0
    diachronic_valid = []
    output_filename = 'r.' + subreddit + '.' + retrieval_type + '.json'
    with open(dirname + output_filename, 'w') as fout:
#         for i in range(len(dates) - 1):
#             after = dates[i] #date farther from present
#             before = dates[i+1] #date closer to present
        epoch_valid = 0 # number of data points in the given time period
        done = False

        while not done:
            count += 1
            query = f"{PUSHSHIFT_ENDPOINT}{retrieval_type}/?subreddit={subreddit}&sort=desc&size={MAX_RETRIEVED_ELEMENTS}" +\
            f"&fields={fields}&q={query_words}&before={before_date}"
            # &before={before}&after={after}

#                 query = PUSHSHIFT_ENDPOINT + retrieval_type + '/?subreddit=' + subreddit + \
#                 '&sort=desc&size=' + str(MAX_RETRIEVED_ELEMENTS) + '&before=' + str(before) + '&fields=' + fields + \
#                 "&q=" + query_words + 

            print(query, 'request #', count)
            sys.stdout.flush()
            try:
                r = requests.get(query)
            except:
                print('exception thrown...')
                time.sleep(5)
                continue

            if r.status_code != 200:
                print('bad response code:', r.status_code)
                if r.status_code == 429:
                    time.sleep(3)
                    continue
#                 after += 86400  # slight change to the query
                continue  # retry

# record a submission/comment only if it's not empty

            for i, element in enumerate(r.json()['data']):
#                 latest = element['created_utc']
                if element['author'] == 'AutoModerator':
                    continue
                if 'created_utc' in element.keys():
                    before_date = element['created_utc']
#                     print(before_date)
                    if before_date < int(EARLIEST_DATE.timestamp()):
                        done = True
                        break
                try:
                    is_valid, formatted = valid_text(element[text_tag])
                except KeyError:
                    continue
                if is_valid:
                    element[text_tag] = formatted
                    json.dump(element, fout)
                    fout.write('\n')
                    total_valid += 1
                    epoch_valid += 1

            if len(r.json()['data']) < MAX_RETRIEVED_ELEMENTS:  # end of data
                done = True
#             else:
#                 before_date
#                 after = latest
        diachronic_valid.append(epoch_valid)
                
    print("SUBREDDIT: r/{}".format(subreddit))
    print(retrieval_type)
    print(text_tag)
    print(total_valid)
#     print(len(diachronic_valid))
    return total_valid, np.zeros(22)#diachronic_valid

In [33]:
x = list(state_sub_map.values())
all_districts = []
for smol in x:
    all_districts.extend(smol)
for d in all_districts:
    if len(re.findall(r"\W", d)) > 0:
        print(d)
state_sub_map

FlushingQueens/
bothell/
skagit/


{'Alabama': ['alabama',
  'Anniston',
  'ArabAlabama',
  'auburn',
  'baldwincountyal',
  'birmingham',
  'blountsville',
  'Calera',
  'CalhounCounty',
  'Cullman',
  'dothan',
  'DothanAL',
  'fairhopeAlabama',
  'FayetteCounty',
  'florenceal',
  'gadsden',
  'gordo',
  'greenvilleal',
  'Hartselle',
  'cityofhoover',
  'HooverAL',
  'huntsvillealabama',
  'JacksonvilleAL',
  'Madisonalabama',
  'marshallcountyal',
  'mobileal',
  'montgomery',
  'mountainbrook',
  'oba',
  'OxfordAL',
  'prattville',
  'Ragland',
  'selma',
  'shoals',
  '251',
  'SummerdaleAL',
  'sylacauga',
  'talladega',
  'ThomasvilleAL',
  'Trussville',
  'tuscaloosa',
  'vestavia',
  'vestaviahills'],
 'Alaska': ['alaska',
  'AleutianIslands',
  'Anchorage',
  'BarrowAK',
  'BethelAK',
  'Cordova',
  'fairbanks',
  'homerAK',
  'juneau',
  'Kenai',
  'ketchikan',
  'kodiak',
  'diomede',
  'MatSu',
  'nome',
  'PalmerAK',
  'Seward',
  'Sitka',
  'skagway',
  'spenardak',
  'sterlingalaska',
  'Talkeetna',
 

In [37]:
retrieval_types = ['submission', 'comment']
# country_subreddits = ['europe', 'spain', 'italy', 'unitedkingdom', 'germany', 'iran', 'japan', 'china', 'singapore',
#               'india', 'korea', 'france', 'unitedstatesofamerica', 'canada', 'onguardforthee', 'coronavirusus', 
#               'canadacoronavirus', 'turkey', 'russia', 'brasil', 'belgium', 'thenetherlands', 'portugal', 'switzerland', 
#               'peru', 'coronavirusuk']
# state_subreddits = ['coronavirusca', 'coronavirusNewYork', 'newyork', 'newjersey', 'massachusetts', 'California',  
#                     'Pennsylvania', 'illinois', 'michigan', 'florida', 'louisiana', 'texas', 'coronavirusAZ']  
fields = {'submission': ['author', 'id', 'selftext', 'permalink', 'title', 'created_utc'], 
          'comment': ['author', 'body', 'created_utc', 'permalink']}
keywords = ['covid', 'covid-19', 'coronavirus', 'corona', 'covid19', "pandemic", '"the%20virus"']
#optionally: pandemic?
dates, epochs = get_dates(datetime.utcnow())

output_filename = 'overall_stats.csv'
diachronic_filename = 'diachronic_stats.csv'
with open(DIRNAME + output_filename, 'a') as fout:
    with open(DIRNAME + diachronic_filename, 'a') as dfout:
        fout.write('Subreddit,Sample Size\n')
        dfout.write('Subreddit, {}'.format(",".join([f"Week {i+1}" for i in range(len(dates)-1)])))
        dfout.write("\n")
        for state in state_sub_map:
            print(state)
            total_valid = 0
            diachronic_valid = np.zeros(len(epochs)-1)
            for sub in state_sub_map[state]:
                if sub[-1] == '/':
                    sub = sub[:-1]
                for r_type in retrieval_types:
                    sub_valid, sub_diachronic_valid = retrieve_subreddit_data(sub, r_type, DIRNAME, fields[r_type], keywords, epochs)
                    diachronic_valid += np.array(sub_diachronic_valid)
                    total_valid += sub_valid
            fout.write(f"{state},{total_valid}\n")
            for date_idx in range(len(dates)-1):
                d_vals = ','.join(map(str, map(int, diachronic_valid)))
                dfout.write(f"{state},{d_vals}\n")


Alabama
https://api.pushshift.io/reddit/search/submission/?subreddit=alabama&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589489053 request # 1
SUBREDDIT: r/alabama
submission
selftext
90
https://api.pushshift.io/reddit/search/comment/?subreddit=alabama&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589489054 request # 1
https://api.pushshift.io/reddit/search/comment/?subreddit=alabama&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1584647474 request # 2
SUBREDDIT: r/alabama
comment
body
1138
https://api.pushshift.io/reddit/search/submission/?subreddit=Anniston&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=158

SUBREDDIT: r/FayetteCounty
comment
body
0
https://api.pushshift.io/reddit/search/submission/?subreddit=florenceal&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589489076 request # 1
bad response code: 429
https://api.pushshift.io/reddit/search/submission/?subreddit=florenceal&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589489076 request # 2
SUBREDDIT: r/florenceal
submission
selftext
1
https://api.pushshift.io/reddit/search/comment/?subreddit=florenceal&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589489080 request # 1
SUBREDDIT: r/florenceal
comment
body
4
https://api.pushshift.io/reddit/search/submission/?subreddit=gadsden&sort=desc&size=1000&fields=author,id,selftext,permalink,title,create

SUBREDDIT: r/montgomery
submission
selftext
4
https://api.pushshift.io/reddit/search/comment/?subreddit=montgomery&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589489102 request # 1
bad response code: 429
https://api.pushshift.io/reddit/search/comment/?subreddit=montgomery&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589489102 request # 2
SUBREDDIT: r/montgomery
comment
body
12
https://api.pushshift.io/reddit/search/submission/?subreddit=mountainbrook&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589489106 request # 1
SUBREDDIT: r/mountainbrook
submission
selftext
0
https://api.pushshift.io/reddit/search/comment/?subreddit=mountainbrook&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|c

SUBREDDIT: r/tuscaloosa
submission
selftext
5
https://api.pushshift.io/reddit/search/comment/?subreddit=tuscaloosa&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589489144 request # 1
SUBREDDIT: r/tuscaloosa
comment
body
26
https://api.pushshift.io/reddit/search/submission/?subreddit=vestavia&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589489144 request # 1
SUBREDDIT: r/vestavia
submission
selftext
0
https://api.pushshift.io/reddit/search/comment/?subreddit=vestavia&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589489145 request # 1
SUBREDDIT: r/vestavia
comment
body
0
https://api.pushshift.io/reddit/search/submission/?subreddit=vestaviahills&sort=desc&size=1000&fields=author,id,selftext,permalink,title,cre

SUBREDDIT: r/kodiak
submission
selftext
0
https://api.pushshift.io/reddit/search/comment/?subreddit=kodiak&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589489179 request # 1
SUBREDDIT: r/kodiak
comment
body
1
https://api.pushshift.io/reddit/search/submission/?subreddit=diomede&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589489180 request # 1
bad response code: 429
https://api.pushshift.io/reddit/search/submission/?subreddit=diomede&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589489180 request # 2
SUBREDDIT: r/diomede
submission
selftext
0
https://api.pushshift.io/reddit/search/comment/?subreddit=diomede&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronaviru

...

SUBREDDIT: r/doorcounty
comment
body
7
https://api.pushshift.io/reddit/search/submission/?subreddit=eauclaire&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589495305 request # 1
SUBREDDIT: r/eauclaire
submission
selftext
1
https://api.pushshift.io/reddit/search/comment/?subreddit=eauclaire&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589495305 request # 1
SUBREDDIT: r/eauclaire
comment
body
3
https://api.pushshift.io/reddit/search/submission/?subreddit=eau_claire&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589495305 request # 1
SUBREDDIT: r/eau_claire
submission
selftext
3
https://api.pushshift.io/reddit/search/comment/?subreddit=eau_claire&sort=desc&size=1000&fields=author,body,created_utc,p

SUBREDDIT: r/lacrossewi
comment
body
35
https://api.pushshift.io/reddit/search/submission/?subreddit=madisonwi&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589495330 request # 1
SUBREDDIT: r/madisonwi
submission
selftext
223
https://api.pushshift.io/reddit/search/comment/?subreddit=madisonwi&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589495332 request # 1
https://api.pushshift.io/reddit/search/comment/?subreddit=madisonwi&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1587094025 request # 2
https://api.pushshift.io/reddit/search/comment/?subreddit=madisonwi&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1585024330 reque

SUBREDDIT: r/riverfallswi
submission
selftext
0
https://api.pushshift.io/reddit/search/comment/?subreddit=riverfallswi&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589495360 request # 1
SUBREDDIT: r/riverfallswi
comment
body
0
https://api.pushshift.io/reddit/search/submission/?subreddit=riverwest&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589495361 request # 1
SUBREDDIT: r/riverwest
submission
selftext
0
https://api.pushshift.io/reddit/search/comment/?subreddit=riverwest&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589495361 request # 1
SUBREDDIT: r/riverwest
comment
body
0
https://api.pushshift.io/reddit/search/submission/?subreddit=rockdalewi&sort=desc&size=1000&fields=author,id,selftext,permalink,tit

bad response code: 429
https://api.pushshift.io/reddit/search/comment/?subreddit=westallis&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589495383 request # 2
SUBREDDIT: r/westallis
comment
body
0
https://api.pushshift.io/reddit/search/submission/?subreddit=westbend&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589495387 request # 1
SUBREDDIT: r/westbend
submission
selftext
0
https://api.pushshift.io/reddit/search/comment/?subreddit=westbend&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1589495387 request # 1
SUBREDDIT: r/westbend
comment
body
0
https://api.pushshift.io/reddit/search/submission/?subreddit=WI_Boonies&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=covid|covid-19|cor

SUBREDDIT: r/yellowstone
comment
body
40


In [140]:
all_files = os.listdir(DIRNAME)
with open(DIRNAME + 'diachronic_state_stats.csv', "w") as dia_file:
    weeks = ",".join([f"Week {i+1}" for i in range(len(dates)-1)])
    dia_file.write(f"State, {weeks}\n")
    for state in state_sub_map:
        if state == 'New York':
            break
        subs = state_sub_map[state]
        state_data = []
        state_json = []
        for sub in subs:
            state_data.extend([file for file in all_files if file.startswith(f"r.{sub}")])
        for file_name in state_data:
            with open(DIRNAME + file_name, 'r') as file:
                state_json.extend([json.loads(line) for line in file.readlines() if line != ''])
        state_json.sort()
        for file_name in state_json:
            
            

#         diachronic = np.bincount(np.digitize(state_json, epochs))
#         d_vals = ','.join(map(str, map(int, diachronic[1:])))
#         dia_file.write(f"{state},{d_vals}\n")
        
        if state == 'California':
            json.dumps(state_json)
        


In [49]:
np.digitize(state_json, epochs)

NameError: name 'state_json' is not defined